In [2]:
import pymongo
import polars as pl

from dltf.utils.tables import table_to_tokens, is_valid_table
from dltf.utils.misc import lowercase_translator, whitespace_translator
from dltf.utils.settings import DefaultPath as dp
from dltf.utils.datalake import DataLakeHandlerFactory

In [3]:
dlhargs = [
    'mongodb',
    'latsnaptab',
    ['sloth.latest_snapshot_tables']
]

dlh = DataLakeHandlerFactory.create_handler(*dlhargs)

In [4]:
data = pl.read_csv(f'{dp.data_path.tests}/latsnaptab/latsnaptab/results/extracted/final_results_k20_q100.csv').filter(pl.col('mode') == 'bag')
data

query_id,result_id,algorithm,mode,sloth_overlap,algorithm_overlap,set_size_q,set_size_r,set_overlap,set_union_size,bag_size_q,bag_size_r,bag_overlap,jaccard_sim,multi_jaccard_sim,containment,overlap_set_sim,proximity,area_ratio,sloth_time(s),set_time(s),bag_time(s)
i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1070758,97645,"""josie""","""bag""",21,21,24,573,21,576,84,609,21,0.036458,0.035176,0.875,0.25,1.0,0.25,0.001,0.0,0.00001
189421,449467,"""josie""","""bag""",8,8,28,1110,6,1132,44,6596,8,0.0053,0.005272,0.214286,0.136364,1.0,0.181818,0.01,0.00001,0.00001
189421,460512,"""josie""","""bag""",8,8,28,371,4,395,44,996,8,0.010127,0.010025,0.142857,0.090909,1.0,0.181818,0.001,0.0,0.0
834709,1129943,"""josie""","""bag""",8,15,24,27,13,38,32,32,15,0.342105,0.254902,0.5416667,0.40625,0.533333,0.25,0.001,0.0,0.0
570689,1097784,"""josie""","""bag""",2,2,16,107,2,121,22,230,2,0.016529,0.01626,0.125,0.090909,1.0,0.090909,0.001,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1484462,513714,"""josie""","""bag""",22,40,72,260,15,317,108,459,40,0.047319,0.045181,0.208333,0.1388889,0.55,0.2037037,0.001,0.0,0.00001
1714748,1105039,"""josie""","""bag""",6,6,17,67,2,82,24,391,6,0.02439,0.02381,0.117647,0.083333,1.0,0.25,0.001,0.0,0.0
376146,906397,"""josie""","""bag""",1,1,58,114,1,171,60,204,1,0.005848,0.005814,0.017241,0.016667,1.0,0.016667,0.001,0.00001,0.00001


In [5]:
qgroups = sorted([(q[0], g.sort(by='sloth_overlap', descending=True)) for q, g in data.group_by(by='query_id')])

In [5]:
len(qgroups)

30

In [6]:
i = 4
doc = dlh.get_table_by_numeric_id(qgroups[i][0])
print(f'i: {i}, table ID: {qgroups[i][0]}, #matches={qgroups[i][1].shape[0]}, #header_rows={len(doc["headers"])}, #rows={len(doc["content"])}, #columns={len(doc["content"][0])}, valid_columns={doc["valid_columns"]}')

i: 4, table ID: 1598208, #matches=20, #header_rows=1, #rows=31, #columns=8, valid_columns=[0, 0, 1, 1, 0, 0, 0, 0]


In [7]:
qgroups[i][1].head()

query_id,result_id,algorithm,mode,sloth_overlap,algorithm_overlap,set_size_q,set_size_r,set_overlap,set_union_size,bag_size_q,bag_size_r,bag_overlap,jaccard_sim,multi_jaccard_sim,containment,overlap_set_sim,proximity,area_ratio,sloth_time(s),set_time(s),bag_time(s)
i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1598208,1196605,"""josie""","""bag""",30,35,45,36,26,55,62,46,35,0.472727,0.320988,0.5777778,0.5652174,0.857143,0.652174,0.001,0.00002,0.0
1598208,706709,"""josie""","""bag""",29,29,45,536,13,568,62,689,29,0.022887,0.022375,0.2888889,0.209677,1.0,0.467742,0.001,0.0,0.00001
1598208,1635739,"""josie""","""bag""",27,27,45,436,11,470,62,696,27,0.023404,0.022869,0.244444,0.177419,1.0,0.435484,0.001,0.00001,0.00001
1598208,48147,"""josie""","""bag""",27,27,45,1012,11,1046,62,1681,27,0.010516,0.010407,0.244444,0.177419,1.0,0.435484,0.003,0.0,0.00001
1598208,1709936,"""josie""","""bag""",27,27,45,269,11,303,62,372,27,0.036304,0.035032,0.244444,0.177419,1.0,0.435484,0.001,0.0,0.00001


In [31]:
pl.DataFrame(doc['content'], orient='row')

column_0,column_1,column_2,column_3,column_4
str,str,str,str,str
"""20""","""David Baer""","""Defense""","""Handler""","""Shippensburg University"""
"""28""","""Mike Baer""","""Defense""","""Handler""","""Drexel University"""
"""13""","""David Brandolph""","""Offense""","""Handler""","""Arcadia University"""
"""5""","""Kevin Chiang""","""Defense""","""Cutter""","""West Chester University"""
"""6""","""Leon Chou""","""Defense""","""Cutter""","""University of Pennsylvania"""
…,…,…,…,…
"""27""","""Jake Rainwater""","""Offense""","""Cutter""","""Dickinson College"""
"""16""","""Arthur Shull""","""Offense""","""Cutter""","""Cornell University"""
"""24""","""David Stoddard""","""Defense""","""Cutter""","""The College of New Jersey"""


In [32]:
bag_q = set(table_to_tokens(doc['content'], doc['valid_columns'], 'bag', None, set(), [lowercase_translator, whitespace_translator]))
docr = dlh.get_table_by_numeric_id(qgroups[i][1][0]['result_id'][0])
bag_r = set(table_to_tokens(docr['content'], docr['valid_columns'], 'bag', None, set(), [lowercase_translator, whitespace_translator]))
doc['_id_numeric'], len(bag_q), len(bag_r), len(bag_q | bag_r), len(bag_q & bag_r), bag_q & bag_r

(1370219,
 96,
 96,
 146,
 46,
 {'Cornell University@#1',
  'Cutter@#1',
  'Cutter@#10',
  'Cutter@#11',
  'Cutter@#12',
  'Cutter@#13',
  'Cutter@#14',
  'Cutter@#2',
  'Cutter@#3',
  'Cutter@#4',
  'Cutter@#5',
  'Cutter@#6',
  'Cutter@#7',
  'Cutter@#8',
  'Cutter@#9',
  'Defense@#1',
  'Defense@#10',
  'Defense@#11',
  'Defense@#12',
  'Defense@#13',
  'Defense@#2',
  'Defense@#3',
  'Defense@#4',
  'Defense@#5',
  'Defense@#6',
  'Defense@#7',
  'Defense@#8',
  'Defense@#9',
  'Handler@#1',
  'Handler@#2',
  'Handler@#3',
  'Handler@#4',
  'Handler@#5',
  'Handler@#6',
  'Handler@#7',
  'Offense@#1',
  'Offense@#10',
  'Offense@#2',
  'Offense@#3',
  'Offense@#4',
  'Offense@#5',
  'Offense@#6',
  'Offense@#7',
  'Offense@#8',
  'Offense@#9',
  'Swarthmore College@#1'})

In [33]:
IDs = {
    # ricerca di tabelle con contenuto identiche/simili
    'contenuto_simile': [1749224, 1484462, 1714748, 551590, 1324441, 1121158, 1216805, 1370219, 1216034, 1749224, 1623113, 1591122, 463713, 376146],
    
    # esempio di pulizia per tabelle sporche? Con token e robe strane in mezzo, tipo "{{nom}}"
    'pulizia_tabelle_sporche': [1249360, 1630678, 612879],
    
    # esempio di ricerca di tabelle con stessi nomi per verifica di coerenza dati
    'verifica_coerenza': [1878225, 1878225, 1598208, 766881, 1195130, 834709, 323280],

    'scartate': [570689, 1394327, 189421, 1347854, 1536126, 737775, 1070758, 461074]
}

### How many tables do we have with the previous tables and the relative result IDs?

In [34]:
query_ids = {qid for k, qlist in IDs.items() if k != 'scartate' for qid in qlist}
len(query_ids)

22

In [35]:
[q for q in qgroups[0][1]['result_id']]

[505572,
 533946,
 674707,
 675588,
 747751,
 691431,
 1016799,
 847565,
 906844,
 831783,
 479749,
 808444,
 837511,
 678351,
 645723,
 674928,
 807451,
 655816,
 974076,
 685388]

In [36]:
ids = {i for qid, qgroup in qgroups if qid in query_ids and qgroup.shape[0] >= 5 for i in qgroup['result_id']}
len(ids | query_ids)

450

In [37]:
ids = ids | query_ids

### Get random tables to complete the datalake

In [40]:
i = 0
n = 0
for doc in dlh.scan_tables():
    i += 1
    if len(doc['headers']) == 1:
        n += 1
    if i == 10000:
        break
n

4287

In [41]:
import random

datalake_size = 1_000
datalake_ids = ids
n_tables = dlh.count_tables()


for doc in dlh.scan_tables():
    if doc['_id_numeric'] not in datalake_ids and is_valid_table(doc['content'], doc['valid_columns']) and len(doc['headers']) == 1 and random.random() < 0.5:
        datalake_ids.add(doc['_id_numeric'])
    if len(datalake_ids) >= datalake_size:
        break
    
    print(f'{doc["_id_numeric"]}/{n_tables}, found {len(datalake_ids)}/{datalake_size} tables', end='\r')

In [42]:
import json

IDs = {
    'all_ids': list(datalake_ids),
    'query_ids': list(query_ids)
}

with open('datalake_ids.json', 'w') as fw:
    json.dump(IDs, fw, indent=4)

IDs

{'all_ids': [4098,
  10,
  4110,
  4111,
  18,
  23,
  1761308,
  29,
  4128,
  4133,
  40,
  1290283,
  4141,
  47,
  48,
  880689,
  4143,
  51,
  4144,
  4156,
  4159,
  49216,
  4169,
  1249360,
  614481,
  80,
  4183,
  90,
  909405,
  471135,
  95,
  98,
  4196,
  105,
  1769579,
  114,
  4213,
  119,
  8313,
  121,
  1695867,
  125,
  684160,
  1532033,
  200834,
  128,
  132,
  4225,
  1216646,
  4231,
  137,
  4236,
  141,
  4237,
  4238,
  144,
  925846,
  860312,
  4248,
  155,
  159,
  160,
  161,
  162,
  163,
  4262,
  4264,
  1245360,
  4282,
  606400,
  1634500,
  528592,
  214,
  1749223,
  1749224,
  1593580,
  1921265,
  241,
  245,
  256,
  257,
  1564930,
  259,
  1749255,
  373000,
  264,
  823562,
  1888522,
  276,
  277,
  280,
  1528093,
  1605917,
  285,
  288,
  289,
  292,
  1216805,
  831783,
  302,
  306,
  314,
  627013,
  327,
  1249611,
  343,
  1716573,
  354,
  905572,
  356,
  98678,
  375,
  377,
  378,
  1081725,
  381,
  382,
  401792,
  384,
  20

### Get all the documents and save them

In [43]:
docs = []

mongoclient = pymongo.MongoClient()
collection = mongoclient.sloth.latest_snapshot_tables

for id in datalake_ids:
   doc = collection.find_one({f'_id_numeric': id})
   docs.append(doc) 

In [44]:
len(docs)

1000

In [45]:
with open('demo_datalake.json', 'w') as fw:
    json.dump(docs, fw)

## Inspect the tables

In [6]:
import tabulate

In [7]:
mongoclient = pymongo.MongoClient()
collection = mongoclient.sloth.latest_snapshot_tables

In [18]:
i = 2

qid = qgroups[i][0]
results = [row for row in qgroups[i][1].select('result_id', 'sloth_overlap').rows()]

In [19]:
results

[(1634500, 2),
 (877051, 1),
 (789308, 1),
 (844194, 1),
 (261573, 1),
 (29533, 1),
 (1998434, 1),
 (906397, 1)]

In [20]:
qdoc = collection.find_one({'_id_numeric': qid})
print(qdoc['_id_numeric'], qdoc['_id'], qdoc['context'])
print(tabulate.tabulate(qdoc['content']))

376146 597.79521 ['Viceroyalty of the Río de la Plata', 'Government | List of viceroys']
--  -----------------------------------------------------------  ------------------------------  --------------------------------------  ----------------  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#   Picture                                                      Name                            Term                                    Designation       Notes
1   File:Pedro de Cevallos.jpg                                   Pedro Antonio de Cevallos       15 October 1777 &ndash; 26 June 1778    1 August 1776     Appointed by Cha

In [24]:
j = 1

rid = results[j][0]
rdoc = collection.find_one({'_id_numeric': rid})
print(rdoc['_id_numeric'], rdoc['_id'], rdoc['context'])
print(tabulate.tabulate(rdoc['content']))

877051 415.119929 ['List of Great Britain by-elections (1790–1800)', 'By-elections | List of MPs elected in the British general election, 179618th Parliament (1796–1800)']
-----------------  ------------------------  ----  -------------------------------  -----------------------------------  -------------------------------------------------------------------------------------------
Date               Constituency              c/u   Former Incumbent                 Winner                               Cause
26 October 1796    Great Yarmouth            c     Lord Charles Townshend           Henry Jodrell                        Death (Murdered)
26 October 1796    Great Yarmouth            c     Stephens Howe                    William Loftus                       Death
26 October 1796    Hereford                  u*    John Scudamore                   John Scudamore II                    Death
26 October 1796    Peterborough              u*    Richard Benyon                   French Laure

In [32]:
from dltf.sloth.sloth import sloth
metrics = []
res = sloth(qdoc['content'], rdoc['content'], verbose=True, complete=True, metrics=metrics)
print('Number of distinct overlaps: ', len(res[0]))
for r in res[0]:
    print(tabulate.tabulate([[x] for x in r[1][0]]))

Detected 1 seed(s) in 0.010602712631225586 seconds.
Generated 0 candidate(s) in 0.0 seconds.
Verified 0 candidate(s) in 0.0 seconds.
Detected 1 largest overlap(s) in 1.5974044799804688e-05 seconds.
Size of the detected largest overlap(s): 1 columns, 1 rows, 1 cells.
Total elapsed time: 0.01163792610168457 seconds.
                 0
0  25 October 1797
Number of distinct overlaps:  1
---------------
25 October 1797
---------------


In [30]:
qdoc['content'][8]

['7',
 'File:Avilés1.jpg',
 'Gabriel de Avilés,\n2nd Marquis of Avilés',
 '14 March 1799 &ndash; 20 May 1801',
 '25 October 1797',
 '']

In [31]:
rdoc['content'][58]

['25 October 1797',
 'Leicestershire',
 'u*',
 'Penn Assheton Curzon',
 'George Anthony Legh Keck',
 'Death']

In [26]:
res

([([(8, 58)], [('25 October 1797',)])],
 [1,
  0.012362480163574219,
  'e',
  None,
  4.5299530029296875e-06,
  0,
  0.0,
  0,
  0.0,
  1,
  1,
  1,
  1,
  0.012433290481567383])

Examples:

--> same topic, 2012 European Athletics Championships - Women's 4x100 metres relay, results FINAL vs ROUND-1
query 766881, result 766450